---

<div class="alert alert-primary alert-info">

# Frozen Lake $4\times4$ と $8\times8$

## Reinforcement Learning

</div>

<div class="alert alert-block alert-success">

- ### Policy-Iteration
    
</div>

---

<img src='frozenlake.jpg' width=1000 height=50/>

---

In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import gym

In [3]:
np.random.seed(1)

<div class="alert alert-primary alert-info">

## Non-slippery version

</div>

---

### Policy Iteration (slippery = False)

---

The transitional probabilities are deterministic in the unslippery version. We can simplify

$
\begin{align}
q_{\pi}(s, a) &:= \sum_{\forall{s'}} p_{ss'}^a ( R_{s'}^a + \gamma \sup_{\forall{a'}} \{ q_{\pi}(s', a') \} ) \\
&= p_{ss'}^a ( R_{s'}^a + \gamma \sup_{\forall{a'}} \{ q_{\pi}(s', a') \} )
\end{align}
$

---

In [4]:
def policy_iteration(env, gamma, epsilon, max_iterations):
    
    LEFT, DOWN, RIGHT, UP, TERMINAL = 0, 1, 2, 3, -1
    
    def policy_evaluation():
        for state in range(env.nS - 1):
            action = policy[state]
            transition_probability, next_state, reward, done = env.P[state][action][0]
            best_next_action_value = np.max(action_values[next_state])
            action_values[state][action] = transition_probability * (reward + gamma * best_next_action_value)
                    
    def policy_improvement():
        for state in range(env.nS - 1):
            best_action_value = 0.0
            for action in range(env.nA):
                transition_probability, next_state, reward, done = env.P[state][action][0]
                best_next_action_value = np.max(action_values[next_state])
                if best_action_value < transition_probability * (reward + gamma * best_next_action_value):
                    best_action_value = transition_probability * (reward + gamma * best_next_action_value)
                    policy[state] = action

    def print_policy(n):
        idx = 0
        for state, action in enumerate(policy):
            idx += 1
            if state == env.nS - 1:
                print('G', end=' ')
            elif action == LEFT:
                print('<', end=' ')
            elif action == DOWN:
                print('v', end=' ')
            elif action == RIGHT:
                print('>', end=' ')
            elif action == UP:
                print('^', end=' ')
            if idx % n is 0:
                print('\n')
  
    policy = np.random.randint(0, 4, size=(env.nS))
    
    action_values = np.zeros((env.nS, env.nA))
    
    env.reset()
    print('Start:')
    env.render()
    
    print('\nInitial random policy:\n')
    if env.nS == 64:
        print_policy(8)
    else:
        print_policy(4)

    iteration = 0
    
    while True:
        iteration += 1
        prev_action_values = action_values.copy()
        policy_evaluation()
        delta = np.fabs(action_values - prev_action_values).max()
        if delta < epsilon * (1 - gamma) / gamma and delta != 0:
            break
        if iteration >= max_iterations:
            break
        policy_improvement()

    print('\nNumber of Iterations: ', iteration)
    print('Delta: ', delta)
    print('\nFinal policy:\n')
    if env.nS == 64:
        print_policy(8)
    else:
        print_policy(4)

---

### $4\times4$

---

In [5]:
env = gym.make('FrozenLake-v0', is_slippery=False)
gamma = 0.999
epsilon = 0.01
max_iterations = 10000

if __name__=='__main__':
    policy_iteration(env, gamma, epsilon, max_iterations)

Start:

SFFF
FHFH
FFFH
HFFG

Initial random policy:

v ^ < < 

^ v ^ v 

^ < < v 

< ^ v G 


Number of Iterations:  10000
Delta:  0.0

Final policy:

v > v < 

v v v v 

> v v v 

< > > G 



---

### $8\times8$

---

In [6]:
env = gym.make('FrozenLake8x8-v0', is_slippery=False)
gamma = 0.999
epsilon = 0.01
max_iterations = 10000

if __name__=='__main__':
    policy_iteration(env, gamma, epsilon, max_iterations)

Start:

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

Initial random policy:

> v > < > v > < 

^ < > < v > > < 

^ ^ v v ^ > < > 

v v v ^ ^ v > v 

v < < v < < v ^ 

^ > v < > ^ ^ > 

v v v ^ < < v ^ 

< > < < v ^ v G 


Number of Iterations:  10000
Delta:  0.0

Final policy:

v v v v v v v v 

v v v > v v v v 

v v v v v > v v 

> > > > v v v v 

> > ^ v v v > v 

v > v > > v ^ v 

v v > ^ < v v v 

> > ^ < > > > G 



---

<div class="alert alert-primary alert-info">

## Slippery when Wet

<img src='Slippery_when_wet.jpg' width=250 height=5/>

</div>

---

$
\begin{align}
q_{\pi}(s, a) &:= \sum_{\forall{s'}} p_{ss'}^a ( R_{s'}^a + \gamma \sup_{\forall{a'}} \{ q_{\pi}(s', a') \} )
\end{align}
$

---
 
 - $\frac{1}{4}$ of the states in the $4x4$ grid are holes. Any randomized starting policy has a only a small probability of reaching the goal state (and ultimately collecting the reward of $1.0$) without falling into the holes.


 - Enhanced policy improvements using intended action lookbacks. Keep track of previous states to estimate true current state.


---

In [7]:
def policy_iteration_slippery(env, gamma, epsilon, max_iterations):
    
    LEFT, DOWN, RIGHT, UP, TERMINAL = 0, 1, 2, 3, -1
    
    def policy_evaluation():
        for state in range(env.nS - 1):
            action = policy[state]
            total_expectation_state_value = 0.0
            observations = env.P[state][action]
            for observation in observations:
                transition_probability, next_state, reward, done = observation
                best_next_action_value = np.max(action_values[next_state])
                total_expectation_state_value += ((reward + gamma * best_next_action_value) * transition_probability)
            action_values[state][action] = total_expectation_state_value
    
    def policy_improvement(n):
        for state in range(env.nS - 1):
            best_action, best_action_value = 1, 0.0
            for action in range(env.nA):
                observations = env.P[state][action]
                action_value, action = 0.0, 0
                for observation in observations:
                    transition_probability, next_state, reward, done = observation
                    if action_value < reward + gamma * transition_probability * np.max(action_values[next_state]):
                        action_value = reward + gamma * transition_probability * np.max(action_values[next_state])
                        if state - 1 == next_state:
                            action = LEFT
                        elif state + 1 == next_state:
                            action = RIGHT
                        elif state + n  == next_state:
                            action = DOWN
                        elif state - n == next_state:
                            action = UP
                if best_action_value < action_value:
                    best_action_value = action_value
                    best_action = action
            policy[state] = best_action
    

    def print_policy(n):
        idx = 0
        for state, action in enumerate(policy):
            idx += 1
            if state == env.nS - 1:
                print('G', end=' ')
            elif action == LEFT:
                print('<', end=' ')
            elif action == DOWN:
                print('v', end=' ')
            elif action == RIGHT:
                print('>', end=' ')
            elif action == UP:
                print('^', end=' ')
            else:
                print('H', end=' ')
            if idx % n is 0:
                print('\n')
  
    policy = np.random.randint(0, 4, size=(env.nS))
    
    action_values = np.zeros((env.nS, env.nA))
    
    env.reset()
    print('Start:')
    env.render()
    
    print('\nInitial random policy:\n')
    if env.nS == 64:
        print_policy(8)
    else:
        print_policy(4)

    iteration = 0
    
    while True:
        iteration += 1
        prev_action_values = action_values.copy()
        policy_evaluation()
        delta = np.fabs(action_values - prev_action_values).max()
        if delta < epsilon * (1 - gamma) / gamma and delta != 0:
            break
        if iteration >= max_iterations:
            break
        if env.nS == 64:
            policy_improvement(8)
        else:
            policy_improvement(4)

    print('\nNumber of Iterations: ', iteration)
    print('Delta: ', delta)
    print('\nFinal policy:\n')
    if env.nS == 64:
        print_policy(8)
    else:
        print_policy(4)

---

### $4\times4$

---

In [8]:
env = gym.make('FrozenLake-v0', is_slippery=True)
gamma = 0.999
epsilon = 0.0001
max_iterations = 100000

if __name__=='__main__':
    policy_iteration_slippery(env, gamma, epsilon, max_iterations)

Start:

SFFF
FHFH
FFFH
HFFG

Initial random policy:

^ < < v 

^ > ^ ^ 

> v < > 

v ^ ^ G 


Number of Iterations:  38
Delta:  9.746847311198348e-08

Final policy:

v > v < 

v v v v 

> v v v 

v > > G 



---

### $8\times8$

---

In [9]:
env = gym.make('FrozenLake8x8-v0', is_slippery=True)
gamma = 0.999
epsilon = 0.0001
max_iterations = 100000

if __name__=='__main__':
    policy_iteration_slippery(env, gamma, epsilon, max_iterations)

Start:

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

Initial random policy:

^ v < ^ v v ^ > 

< < > > ^ ^ ^ ^ 

v ^ ^ < > > > < 

< ^ ^ v v v ^ < 

< v v > < v > > 

< ^ v ^ ^ v < ^ 

< ^ < > < ^ ^ v 

< v < < > < v G 


Number of Iterations:  129
Delta:  9.596370285913647e-08

Final policy:

> > > > > > v v 

^ ^ ^ > > > > v 

^ ^ ^ v ^ > > v 

^ ^ < < ^ v > v 

^ ^ < v > > > v 

^ v v > > v v v 

^ v > ^ v v v v 

^ < < v > > > G 



---